In [1]:
def do_setup():
    # How to download competition data to temp folder(data) 
    # unzip it there, then symlink it like its a subdir
    # NOTE: make sure kaggle.json is in /root/.kaggle/

    #remove original symlink from this directory
    !rm ./data
    
    #remove old setup files
    !rm setup.sh

    #create temp holder
    !mkdir /root/data

    #symlink it
    !ln -s /root/data ./data

    #download competition data to temp data folder
    !cd ./data;kaggle competitions download -c paddy-disease-classification

    #unzip it, -q is silent
    !cd ./data;unzip -q paddy-disease-classification.zip

    #setup dotfiles
    !wget "https://raw.githubusercontent.com/CNUClasses/dotfiles/master/setup.sh";chmod 766 setup.sh;source ./setup.sh  
    
import os
if(not os.path.exists('./data/train_images')):
   !chmod 600 /root/.kaggle/kaggle.json
   do_setup()  

In [2]:
#install needed libraries
try: from path import Path
except ModuleNotFoundError:
    !pip install path --quiet
    from path import Path
try: import timm
except ModuleNotFoundError:
    !pip install timm --quiet
    import timm
try: import optuna
except ModuleNotFoundError:
    !pip install optuna --quiet
    import optuna    

In [3]:
import torch
import pandas as pd
import torch.optim as optim
from tqdm.auto import tqdm
import sklearn
from sklearn.model_selection import train_test_split
sklearn.__version__
import paddy_funcs_classes as pfc

# autoreload extension
%load_ext autoreload
%autoreload 2

### CFG file

In [4]:
#from https://www.kaggle.com/code/hinepo/transfer-learning-with-timm-models-and-pytorch
class CFG:
    train_path='./data/train_images'   #get train and validation Datasets from here
    test_path='./data/test_images'     #get test set from here
    csv_path='./data/train.csv'
    
    ### split train and validation sets
    split_fraction = 0.2

    ##dataloader
    drop_last=False
    
    ### model
    model_name = 'resnet26d'#'convnext_small_in22k' # ## 'resnet50' # 'resnet34', 'resnet200d', 'efficientnet_b1_pruned', 'efficientnetv2_m', efficientnet_b7 ...  

    #get a subset of data to work on(start with True until all experiments done
    #and want to train on entire dataset
    subsample_data=False
    BATCH_SIZE= 8 if subsample_data else 32
    N_EPOCHS = 4 if subsample_data else 50
    # print_freq = 2 

    momentum=0.9
    
    ### set only one to True
    save_best_loss = False
    save_best_accuracy = True

    ### optimizer
    # optimizer = 'adam'
    # optimizer = 'adamw'
    optimizer = 'rmsprop'  
    # LEARNING_RATE=0.00001
    LEARNING_RATE_MIN=0.004
    LEARNING_RATE_MAX=0.01

    random_seed = 42

### Get data

In [5]:
df =pd.read_csv(CFG.csv_path)
# df.head()

### EXPERIMENT ON SUBSET OF DATA?

In [6]:
if CFG.subsample_data:
    #stratify dataframe by multiple columns (see Paddy-Multi.ipynb)
    # df, _ = train_test_split(df, test_size=0.5, random_state=0, stratify=df[['label', 'variety']])

#     #get a small dataset to train on
    df=df.iloc[:500,:]

    print(df.shape)

In [7]:
# #get a list of files
trn_val_files=pfc.get_fls(CFG.train_path)  
tst_files=pfc.get_fls(CFG.test_path) 
print(f'Len train={len(trn_val_files)}, length test={len(tst_files)}')

Len train=10407, length test=3469


### Split data

In [8]:
#stratify dataframe by multiple columns (see Paddy-Multi.ipynb)
train, val = train_test_split(df, test_size=CFG.split_fraction, random_state=0, stratify=df[['label', 'variety']])
len(train) + len(val)

10407

### Get a list of transforms that the original model used

In [9]:
cfg=timm.data.resolve_data_config({}, model=CFG.model_name, verbose=True)
cfg

{'input_size': (3, 224, 224),
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'crop_pct': 0.875}

### Create Datasets

In [10]:
#get transforms
train_transforms, val_transforms=pfc.get_transforms(cfg)

trn_dataset = pfc.MultiTaskDatasetTrain(CFG.train_path,df=train,transforms=train_transforms) #use train df
val_dataset = pfc.MultiTaskDatasetTrain(CFG.train_path,df=val,transforms=val_transforms)   #use val df
test_dataset= pfc.MultiTaskDatasetTest(CFG.test_path, transforms=val_transforms)            #test set

print(f'Number train images={len(trn_dataset)}, number validation={len(val_dataset)}, and number test images={len(test_dataset)}')

if(not CFG.subsample_data):
    assert(len(trn_dataset)+len(val_dataset)==len(trn_val_files))
    assert(len(test_dataset)==len(tst_files))

Number train images=8325, number validation=2082, and number test images=3469


### Create DataLoaders

In [11]:
from torch.utils.data import DataLoader
trn_dl=DataLoader(trn_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=2,drop_last=CFG.drop_last) #drop last to avoid crash
val_dl=DataLoader(val_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=2,drop_last=CFG.drop_last)

#### Sizes from DataLoader

In [12]:
import matplotlib.pyplot as plt
imgs,lbls,varieties = next(iter(val_dl))
print(f"Feature batch shape: {imgs.size()}")
print(f"Labels batch shape: {lbls.size()}")
print(f"Varieties batch shape: {varieties.size()}")

Feature batch shape: torch.Size([32, 3, 224, 224])
Labels batch shape: torch.Size([32])
Varieties batch shape: torch.Size([32])


#### Show the images <mark> is this correct?

In [13]:
def show_some_images():
    #look at original image and new one
    from PIL import Image
    import numpy as np
    img1,lbl,variety = trn_dataset[0]

    # original image (PIL)
    Image.open(trn_dataset.files[0]).show()

    #see returned as a PIL image
    import torchvision.transforms as T
    transform = T.ToPILImage()
    transform(img1).show()

    #show as raw tensor
    def show(img):
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation='nearest')
    show(img1)

    mpr=mapper(df)   
    print(f"Label: {mpr.i_to_label[lbl]}")
    print(f"Variety: {mpr.i_to_variety[variety]}")
    
# show_some_images()

### Define Multi Head Model (2 output params)

In [29]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR  

class DiseaseAndTypeClassifier(nn.Module):
    def __init__(self,tmodel):
        '''
        tmodel: pretrained model
        ex:
        model_name='resnet26d'
        tmodel=timm.create_model(model_name, pretrained=True)
        m1=DiseaseAndTypeClassifier(tmodel)
        
        '''
        super().__init__()
        self.m = tmodel
        #add an extra layer
        # self.m.fc=nn.Linear(in_features=self.m.get_classifier().in_features,out_features=512, bias=False)
        self.l1=nn.Linear(in_features=self.m.get_classifier().out_features, out_features=10, bias=False)  #rice type
        self.l2=nn.Linear(in_features=self.m.get_classifier().out_features, out_features=10, bias=False)  #disease
        
    def forward(self,x):       
        x=F.relu(self.m(x)) 
        label=self.l1(x)  #disease type
        variety=self.l2(x)  #variety
        return label,variety

def get_model(model_name, min_lr,max_lr,num_epochs,trn_dl):
    '''
    gets a timm model and wraps in a  DiseaseAndTypeClassifier class
    min_lr: minimum learning rate for OneCycleLR
    max_lr: maximum learning rate for OneCycleLR
    num_epochs: how many epochs to train for
    trn_dl: TRAIN dataloader-used by OneCycleLR to calculate correct number of steps
    cfg:
    '''
    #create the timm model
    tmodel=timm.create_model(model_name, pretrained=True, num_classes=512,global_pool='catavgmax') 

    #and pass it to DiseaseAndTypeClassifier
    m1=DiseaseAndTypeClassifier(tmodel)

    #create optimizer
    optimizer=optim.SGD(m1.parameters(),min_lr, momentum=CFG.momentum)

    #create a learning rate scheduler
    scheduler = OneCycleLR(optimizer, 
                max_lr = max_lr, # Upper learning rate boundaries in the cycle for each parameter group
                steps_per_epoch=int(len(trn_dl)),epochs=num_epochs,
                anneal_strategy = 'cos') # Specifies the annealing strategy

    #create the learner that will train the model
    return pfc.Learner(m1,scheduler,optimizer)


### Try and find a good learning rate, Optuna and Fastai LR finder

#### Load Optuna

In [16]:
%%time
import optuna

def objective(trial,trn_dl=trn_dl,val_dl=val_dl):
    
    #these are the parameters I want to optimize
    lr = trial.suggest_float("lr", 1e-5, 1e-1)
    
    #get a model
    lrn=get_model(CFG.model_name,min_lr=lr,max_lr=10*lr, num_epochs=1,trn_dl=trn_dl)

    #train the model
    lrn._trn_epoch(trn_dl)
    
    #see how good it is
    acc_labl,_ = pfc.get_accuracy(lrn.m,val_dl, verbose=False)
    
    #get the cross validation score
    return acc_labl

#object that will optimize the objective
study = optuna.create_study(direction='maximize')

#start optimizing, go for 50 rounds
study.optimize(objective, n_trials=3)

trial = study.best_trial

print('best accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 11.7 µs


#### Fastai LR finder

In [17]:
import fastcore
# learn.lr_find()

### Load and Train Model

In [38]:
%%time
lrn=get_model(model_name=CFG.model_name,min_lr=CFG.LEARNING_RATE_MIN,max_lr=CFG.LEARNING_RATE_MAX, num_epochs=CFG.N_EPOCHS,trn_dl=trn_dl)

CPU times: user 366 ms, sys: 68.1 ms, total: 434 ms
Wall time: 327 ms


In [ ]:
%%time
num_epochs=15

#train the model
lrn.learn(trn_dl,val_dl,num_epochs=num_epochs)

train:err_rate_labels=0.48,   label_loss=1.41,  err_rate_varieties=0.27   varieties_loss=0.86
valid:err_rate_labels=0.98,   label_loss=5.66,  err_rate_varieties=0.99   varieties_loss=6.98
Epoch 0found and saving better model

train:err_rate_labels=0.28,   label_loss=0.81,  err_rate_varieties=0.16   varieties_loss=0.47
valid:err_rate_labels=0.98,   label_loss=7.22,  err_rate_varieties=0.98   varieties_loss=6.98
train:err_rate_labels=0.23,   label_loss=0.68,  err_rate_varieties=0.12   varieties_loss=0.35
valid:err_rate_labels=0.99,   label_loss=8.40,  err_rate_varieties=0.99   varieties_loss=8.83


### Lets plot the cyclic learning rate

In [ ]:
import matplotlib.pyplot as plt
plt.plot(lrn.lrs)

## Predict on the val set 

In [ ]:
type(lrn)

In [ ]:

    
get_accuracy(lrn.m,val_dl)
    

## Generate submission

In [ ]:
#generate sorted list of files
test_dataset=MultiTaskDatasetTest(CFG.test_path, transforms=val_transforms) 
test_dataset.files.sort()

#generate a non_shuffle dataloader
tst_dl=DataLoader(test_dataset, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=1,drop_last=False)

#where is the model
device = next(m1.parameters()).device.type

#get model predictions
labels=[]
for imgs in tst_dl:
    imgs = imgs.to(device)

    # forward + backward + optimize
    pred_lbls,_ = lrn.m(imgs)
    
    #get the max index
    pred_lbls=torch.argmax(pred_lbls,dim=1).tolist()
    labels=labels+pred_lbls   

#convert indexes to labels
new_labels=[mpr.i_to_label[i] for i in labels]
# get a list of files
files=[fle.split('/')[-1] for fle in test_dataset.files]   

with open('./submission.csv','w') as fle:
    fle.write('image_id,label')
    for i in range(len(files)):
        fle.write('\n'+files[i]+','+new_labels[i])

## Zip And Upload to Kaggle


In [ ]:
!zip ./submission.zip ./submission.csv
!kaggle competitions submit -c paddy-disease-classification -f submission.zip -m "Message"

## Experiment

In [ ]:
#where is this model located?
# tmodel1=timm.create_model(model_name, pretrained=True, num_classes=13,global_pool='catavgmax')  #going to replace the 13
# next(tmodel1.parameters()).device.type

#     def _one_epoch(self,dl,training):
#         running_loss_labels = 0.0
#         running_loss_varieties = 0.0
#         running_err_rate_labels =0.0
#         running_err_rate_varieties =0.0
#         is_trn=self.m.training
        
#         # for i, data in (enumerate(tqdm(dl), 0)):
#         print()
#         for i, data in (enumerate(dl)):

#             # get the inputs, labels is tuple(label, variety)
#             imgs,lbls,varietys = data[0].to(self.device),data[1].to(self.device),data[2].to(self.device)
           
#             if training:
#                 # zero the parameter gradients
#                 self.optimizer.zero_grad()
 
#             # forward + backward + optimize
#             pred_lbls,pred_varieties = self.m(imgs)
             
#             loss_labels , loss_varieties = self.criterion(pred_lbls,pred_varieties, lbls,varietys)
            
#             if training:
#                 #see https://stackoverflow.com/questions/46774641/what-does-the-parameter-retain-graph-mean-in-the-variables-backward-method
#                 loss_labels.backward(retain_graph=True)
#                 loss_varieties.backward()

#             running_loss_labels+=loss_labels.item()
#             running_loss_varieties+=loss_varieties.item()
#             running_err_rate_labels+=error_rate(pred_lbls,lbls)
#             running_err_rate_varieties+=error_rate(pred_varieties,varietys)
 
#             #adjust weights
#             self.optimizer.step()

#             if (i%CFG.print_freq==0):
#                 state='train' if is_trn else 'valid' 
#                 print(f'{state}:err_rate_labels={(running_err_rate_labels/(i)):.2f},   label_loss={(running_loss_labels):.2f},   err_rate_varieties={(running_err_rate_varieties/(i)):.2f}   varieties_loss={(running_loss_varieties):.2f},    ', end='\r', flush=True)
#                 running_loss_labels = 0.0
#                 running_loss_varieties = 0.0
        

In [ ]:
# m1.train()
# m1.eval()
# print(f'{m1.training}, {m1.m.training}, {m1.m.fc.training}')

#check shapes
# (next(iter(trn_dl))[-2:])

# m1.eval()
# m1(trn_dataset[0][0].unsqueeze(0))

 #calls forward
# m1(next(iter(trn_dl))[0]) 
# m1(val_dataset[0].unsqueeze(0))

# m1(torch.randn(1,3,224,224))



# criterion=DiseaseAndTypeClassifierLoss()
# optimizer = optim.SGD(m1.parameters(), lr=0.001, momentum=0.9)
# num_epochs=3
# numb_batches_between_prints=1

        
        
# for epoch in range(num_epochs):  # loop over the dataset multiple times

#     running_loss_labels = 0.0
#     running_loss_varieties = 0.0
#     num_batches=len(trn_dl)/trn_dl.batch_size
#     m1.train()
#     for i, data in (enumerate(tqdm(trn_dl), 0)):

#         # get the inputs, labels is tuple(label, variety)
#         imgs,lbls,varietys = data[0].to(device),data[1].to(device),data[2].to(device)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         pred_lbls,pred_varieties = m1(imgs)
        
#         #loss
#         # loss = criterion(pred_lbls,pred_varieties, lbls,varietys)
#         # loss.backward()
#         loss_labels , loss_varieties = criterion(pred_lbls,pred_varieties, lbls,varietys)
#         loss_labels.backward()
#         loss_varieties.backward()
        
#         running_loss_labels+=loss_labels.item()
#         running_loss_varieties+=loss_varieties.item()
        
#         #adjust weights
#         optimizer.step()
        
#         if i%numb_batches_between_prints==0:
#             if(
#             print(f'label_loss={running_loss_labels/(numb_batches_between_prints*batch_size):.2f}, \
#                   varieties_loss={running_loss_varieties/(numb_batches_between_prints*batch_size):.2f}', end='\r', flush=True)
#             running_loss_labels = 0.0
#             running_loss_varieties = 0.0
    
#     # m1.eval()
#     # num_batches=len(val_dl)/batch_size
#     # for i, data in (enumerate(tqdm(val_dl), 0)): 

        
        
 

# # print('Finished Training')

In [ ]:
# a=np.nan
# a

# import math
# assert (math.isnan(a))

In [ ]:
# m1=timm.create_model('resnet26d', pretrained=True, num_classes=10,global_pool='catavgmax')  #going to replace the 13
# m2=timm.create_model(CFG.model_name, pretrained=True, num_classes=10,global_pool='catavgmax')  #going to replace the 13

# def get_fc(m):
#     #when iterator is exhausted, item will hold last layer
#     for item in m.children():
#         pass
#     print(type(item))
#     if(isinstance(item,torch.nn.modules.linear.Linear)):
#         print('Its linear')
#     else:
#         item= get_fc(item)   
#     return item

# fcl=get_fc(m1)
# m2.head.fc=nn.Sequential(
#             nn.Linear(in_features=m1.get_classifier().in_features,out_features=512, bias=False),
#             nn.ReLU())



# m11=DiseaseAndTypeClassifier(m1)
# m22=DiseaseAndTypeClassifier(m2)

# #convnext has no m1.fc, get its head this way
# tmodel.head.fc